# Реализация алгоритмов

### Импорт необходимых библиотек

In [1]:
import sys

sys.path.insert(1, "../")

import os
import pathlib
import collections

import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score,
)

from src.data.dataset_filenames import extract_type_face
from src.data.utils import is_non_zero_file, get_directory_file_list, resize_img
from src.features import (
    haralick_rdwt_features,
    dispersion_feature,
    mean_feature,
    median_feature,
    maximum_feature,
    minimum_feature,
    fraction_pixels_for_interval_feature,
    scatter_feature,
    hist_feauture,
    hist_gradient_feauture,
    ratio_upper_lower_parts_feature,
)
from src.features.utils import apply_feature_funcs


%matplotlib inline

### Загрузка изображений лиц

In [2]:
path_to_dataset = pathlib.Path(
    pathlib.Path.home(),
    "Projects",
    "ml",
    "datasets",
    "FACE_ANTISPOOFING",
    "FACE_ANTISPOOFING_FACES",
)
path_to_dataset_rgb_faces = path_to_dataset / "rgb"
path_to_dataset_th_faces = path_to_dataset / "thermal"

#### Функция для загрузки изображений датасета с мета-информацией

In [3]:
def load_dataset_objects(path, img_size=(312, 312), cv_mode=cv2.COLOR_BGR2RGB):
    result = {"filenames": [], "images": [], "subclasses": [], "classes": []}

    for root, dirs, files in os.walk(path):
        for filename in files:
            if not filename.startswith("."):
                result["filenames"].append(filename)
                
                image = cv2.imread(os.path.join(root, filename))
                image = cv2.cvtColor(image, cv_mode)
                image = resize_img(image, img_size)
                result["images"].append(image)
                
                result["subclasses"].append(extract_type_face(filename))
                result["classes"].append(int("a" in result["subclasses"][-1]))

    return result

#### Загрузка RGB-изображений лиц

In [4]:
rgb_data = load_dataset_objects(path_to_dataset_rgb_faces)

In [5]:
# plt.imshow(rgb_data["images"][0])

#### Загрузка Thermal-изображений лиц

In [6]:
th_data = load_dataset_objects(path_to_dataset_th_faces, cv_mode = cv2.COLOR_BGR2GRAY)

In [7]:
# plt.imshow(th_data["images"][0], cmap="gray")

### Функция для расчета метрик

In [44]:
def FAR_score(y_true, y_pred):
    y_true = y_true.astype(bool)
    y_pred = y_pred.astype(bool)
    return sum(y_pred & ~y_true) / len(y_true[np.where(y_true == False)])

In [45]:
def FRR_score(y_true, y_pred):
    y_true = y_true.astype(bool)
    y_pred = y_pred.astype(bool)
    return sum(~y_pred & y_true) / len(y_true[np.where(y_true == True)])

In [47]:
def HTER_score(y_true, y_pred):
    far = FAR_score(y_true, y_pred)
    frr = FRR_score(y_true, y_pred)
    hter=(far+frr)/2
    return hter

In [74]:
def get_metric_values(y_true, y_pred, pos_label=1):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="binary", pos_label=pos_label),
        "recall": recall_score(y_true, y_pred, average="binary", pos_label=pos_label),
        "f1": f1_score(y_true, y_pred, average="binary", pos_label=pos_label),
        "roc_auc": roc_auc_score(
            y_true,
            y_pred,
            average="weighted",
        ),
        "hter": HTER_score(y_true, y_pred)
    }

In [78]:
measured_metrics = pd.DataFrame(
    {"Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC", "HTER"]}
)

### Алгоритм RDWT-Haralick-SVM (for thermal)

In [8]:
X = []
y = th_data["classes"].copy()

In [9]:
for img in th_data["images"]:
    X.append(haralick_rdwt_features(img))

In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
svc_lin = SVC(kernel="linear", C=1e2)
svc_lin.fit(X_train_scaled, y_train)
y_pred = svc_lin.predict(X_test_scaled)

In [80]:
measured_metrics["RDWT-Haralick-SVM (for thermal)"] = get_metric_values(y_test,y_pred).values()

In [81]:
measured_metrics

,Metric,RDWT-Haralick-SVM (for thermal)
0,Accuracy,1.0
1,Precision,1.0
2,Recall,1.0
3,F1,1.0
4,ROC-AUC,1.0
5,HTER,0.0
